In [1]:
## This notebook will provide example implementations of key 
## operations in the iRODS protocol

In [2]:
## We'll be doing this from scratch, so all imports will come from 
## the Python standard library
import socket
import struct
import base64
import json
import hashlib
import time
import enum
import xml.etree.ElementTree as ET
from enum import Enum

In [3]:
## This tutorial assumes you have deployed iRODS in Docker using
## the script stand_it_up.py from the iRODS Testing Environment, 
## which can be found on Github here: https://github.com/irods/irods_testing_environment
## To find the IP address associated with your Docker container, you can run this one-liner:
## docker inspect   -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' ubuntu-2004-postgres-1012_irods-catalog-provider_1
## Otherwise, if want to try this out on a real-world zone, insert that zone's hostname here.
HOST = "172.19.0.3"

In [4]:
PORT = 1247 ## This is the standard iRODS port
MAX_PASSWORD_LENGTH = 50 ## This constant comes 
                         ## from the internals 
                         ## of the iRODS server

In [5]:
## First, we're going to write a small library of functions that do some 
## of the dirty work. 
## Feel free to skip to cell 15, where we start using this library to send
## and read messages, referring to this part to figure out how
## the part you're interested in was implemented.

In [6]:
## We can define these in an enum since 
## header types are a closed class and are not sensitive to any
## particular API.
class HeaderType(Enum):
    RODS_CONNECT = "RODS_CONNECT"
    RODS_DISCONNECT = "RODS_DISCONNECT"
    RODS_API_REQ = "RODS_API_REQ"
    RODS_API_REPLY = "RODS_API_REPLY"
    RODS_VERSIN = "RODS_VERSION"

def header(header_type: HeaderType, msg: bytes, error_len=0, bs_len=0, int_info=0):
    return f"""
        <MsgHeader_PI>
            <type>RODS_CONNECT</type>
            <msgLen>{len(msg)}</msgLen>
            <errorLen>{error_len}</errorLen>
            <bsLen>{bs_len}</bsLen>
            <intInfo>{int_info}</intInfo>
        </MsgHeader_PI>
        """.replace(' ', '').replace('\n', '').encode('utf-8') ## The protocol is whitespace-insensitive,
                                                               ## but I removed them here for cleanliness
                                                               ## and efficiency for when this gets pushed
                                                               ## through the pipe.

In [13]:
def send_header(header, sock):
    header_len = int.to_bytes(len(header), byteorder='big', length=4) ## The first part of all iRODS messages
                                                                      ## must be 4 bytes indicating how long
                                                                      ## the header is in bytes. These bytes
                                                                      ## and the entire integer must be transmitted
                                                                      ## in big-endian order
    print(header_len)
    sock.sendall(header_len)
    sock.sendall(header)
    
def send_msg(msg, sock) -> None:
    sock.sendall(msg)
    
def recv(sock) -> [ET, ET]:
    header_len = int.from_bytes(sock.recv(4))
    header = ET.fromstring(
        sock.recv(header_len).decode("utf-8")
    )
    msg = ET.fromstring(sock.recv(
        int(header.find("msgLen").text)
    ).decode("utf-8"))
    
    return header, msg

In [14]:
class IrodsProt(Enum):
    NATIVE_PROT = 0
    XML_PROT = 1

## Now, let's start the connection process. First, we need an easy way to create the StartupPack.
def startup_pack(irods_prot=IrodsProt.XML_PROT,reconn_flag=0, connect_cnt=0,proxy_user=None,proxy_rcat_zone=None,
                      client_user="rods", client_rcat_zone="tempZone", rel_version="4.3.0", 
                      api_version="d", ## This MUST ALWAYS be "d." This value has been hardcoded into iRODS
                                       ## since very early days.
                      option=None ## This option controls, among other things,whether SSL negotiation is required.
                ) -> str:
    return f"""
    <StartupPack_PI>
             <irodsProt>{irods_prot}</irodsProt>
             <reconnFlag>{reconn_flag}</reconnFlag>
             <connectCnt>{connect_cnt}</connectCnt>
             <proxyUser>{proxy_user or client_user}</proxyUser>
             <proxyRcatZone>{proxy_rcat_zone or client_rcat_zone}</proxyRcatZone>
             <clientUser>{client_user}</clientUser>
             <clientRcatZone>{client_rcat_zone}</clientRcatZone>
             <relVersion>rods{rel_version}</relVersion>
             <apiVersion>{api_version}</apiVersion>
             <option>{option}</option>
    </StartupPack_PI>
    """.replace(" ", "").replace("\n", "").encode("utf-8")

In [15]:
## We're going to be sending raw bytes over a socket, so let's create one
## If at some point the Notebook stops working, remember
## to manually close the socket. 
conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
conn.connect((HOST, PORT)) 

In [16]:
sp = startup_pack()
sp

b'<StartupPack_PI><irodsProt>IrodsProt.XML_PROT</irodsProt><reconnFlag>0</reconnFlag><connectCnt>0</connectCnt><proxyUser>rods</proxyUser><proxyRcatZone>tempZone</proxyRcatZone><clientUser>rods</clientUser><clientRcatZone>tempZone</clientRcatZone><relVersion>rods4.3.0</relVersion><apiVersion>d</apiVersion><option>None</option></StartupPack_PI>'

In [17]:
h = header(HeaderType.RODS_CONNECT, sp)
h

b'<MsgHeader_PI><type>RODS_CONNECT</type><msgLen>343</msgLen><errorLen>0</errorLen><bsLen>0</bsLen><intInfo>0</intInfo></MsgHeader_PI>'

In [18]:
send_header(h, conn)
send_msg(sp, conn)

b'\x00\x00\x00\x84'


In [19]:
## In this Version_PI, status of 0 lets us know that negotiation has been successful.
h, msg = recv(conn)
ET.dump(h)
ET.dump(msg)

<MsgHeader_PI>
<type>RODS_VERSION</type>
<msgLen>182</msgLen>
<errorLen>0</errorLen>
<bsLen>0</bsLen>
<intInfo>0</intInfo>
</MsgHeader_PI>
<Version_PI>
<status>0</status>
<relVersion>rods4.3.0</relVersion>
<apiVersion>d</apiVersion>
<reconnPort>0</reconnPort>
<reconnAddr />
<cookie>400</cookie>
</Version_PI>


In [ ]:
## Next up, we need to authenticate using our API of choice. 
## Since this is a basic cookbook for 4.3.0, we'll be using the new 
## auth framework's port of native authentication.

In [20]:
conn.close()